In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
import glob
import os
from random import shuffle

#preprocessor to load your documents
#pass the train and the test set to the below funciton to create dataset of postive and negative data

def pre_process_data(file_path):
    positive_path = os.path.join(file_path, 'pos')
    negative_path = os.path.join(file_path, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []

    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
    for filename in glob.glob(os.path.join(negative_path,"*.txt")):
        with open(filename) as f:
            dataset.append((neg_label, f.read()))
    shuffle(dataset)

    return dataset
train_dataset = pre_process_data('/home/muhammed/Documents/nlpia_codes/ch7/aclImdb/train')
test_dataset = pre_process_data('/home/muhammed/Documents/nlpia_codes/ch7/aclImdb/test')
dataset = train_dataset + test_dataset
print(dataset[0])
#

Using TensorFlow backend.
/home/muhammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/muhammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/muhammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/muhammed/anaconda3/lib/python3.7/site-packag

(0, 'Kathryn Bigelow and Mark Boal are already preparing a sequel about a young US corporal in Afghanistan. He also happens to be a highly-qualified surgeon and is roaming freely around Kabul, operating on wounded NATO soldiers. On a particularly difficult mission, he casually picks up a sniper rifle and shoots Osama Bin Laden from a distance of about 3000 yards. He is then finally promoted to sergeant, but is unable to decide between a sniper and surgeon career, so he quits from the Army altogether. One year later, frustrated with civilian life, he joins the Navy and the last scene shows him proudly wearing a white uniform.')


In [3]:
#vectorizer and tokenizer
#tokenizer is to tokenize the document, into sentences and the sentences into words (tokens)
#vectorizer is to find the word2vce representation of each token from the tokenization phase
#note here we used GoogleNews word2vec

from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
from nlpia.loaders import get_data

EMBEDDING_FILE = '/home/muhammed/Documents/nlpia_codes/GoogleNews-vectors-negative300.bin' # from above
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
   
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        #the document is tokenized into sentences and sentences into tokens
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except:
                pass #No matching token in the Google w2v
        vectorized_data.append(sample_vecs)
    return vectorized_data



/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:87: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:137: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:159: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:167: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly 

In [6]:
#collect the targe

def collect_expected(dataset):
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)


In [7]:
split_point = int(len(vectorized_data)*.8)
x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [13]:
maxlen = 400
#the maximum length for sentence is 400 tokens any longer cut, shorter padd
batch_size = 32
embedding_dims = 300
#the number of dimensions in the embedding vector
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2


In [11]:
def pad_trunc(data,maxlen):
    #pad dataset with zero embedding vectors and truncate the longer one
    new_data = []
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [14]:
x_train = pad_trunc(x_train, maxlen)
x_test  = pad_trunc(x_test, maxlen)

#then reshape the x_train using numpy so keras will be happy with it

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)


MemoryError: Unable to allocate 35.8 GiB for an array with shape (40000, 400, 300) and data type float64

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D,Dense

model = Sequential()
model.add(Conv1D(filters,
                kernel_size,
                padding = 'valid',
                activation = 'relu',
                strides = 1,
                input_shape = (maxlen, embedding_dims)))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#the optimizer
#the loss function is what the nnt is trying to minimize
#the optimizer is the algorthim which will minimize the loss and modify the weights

model.compile(loss='binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

#the fit method is which fit the data
model.fit(x_train,y_train, batch_size = batch_size,
         epochs  = epochs,
         validation_data=(x_test, y_test))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-05-15 01:20:36,907 WARNING:     tensorflow:323:            new_func From /home/muhammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model_structure = model.to_json()
with open('cnn_model.json', 'w') as json_file:
    json_file.write(model_structure)
model.save_weights('cnn_weights.h5')